# Задание 1.

## Реализация алгоритма SymSpell

In [8]:
import re
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from collections import Counter
from string import punctuation
from threegrams import get_threegrams_from_text

start = '<start>'
end = '<end>'


punctuation += "«»—…“”"
punct = set(punctuation)
import gzip
import csv

from sklearn.metrics import classification_report

bad = open('mistakes.txt').read().splitlines()
true = open('correct.txt').read().splitlines()

# corpus = open('corpus_5000.txt', 'w')
# with gzip.open('lenta-ru-news.csv.gz', 'rt') as archive:
#     reader = csv.reader(archive, delimiter=',', quotechar='"')
#     for i, line in enumerate(reader):
#         if i < 5000: 
#             corpus.write(line[2].replace('\xa0', ' ') + '\n')

def normalize(text):
    normalized_text = [(word.strip(punctuation)) for word \
                       in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


corpus = []
for text in open('corpus_5000.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

N = sum(WORDS.values())
def P(word, N=N):
    return WORDS[word] / N

vocabulary1 = {} 
for sent in corpus:
    for word in sent:
        if word not in vocabulary1:
            vocabulary1[word] = P(word)
        else:
            vocabulary1[word] += 1

vocabulary2 = {}  
for key in vocabulary1.keys():
    for sym in key:
        word_with_del = key.replace(sym, '')
        if word_with_del in vocabulary2:
            vocabulary2[word_with_del].append(key)
        else:
            vocabulary2[word_with_del] = [key]

def algoritm(word):

    wordforms = []
    wordforms_keys = []
    candidates = {}
    best_candidate = []
    
    if word in vocabulary1:  #если слово есть в словаре норм слов, возвращаем его
        return word
    else:  #иначе – удаляем по 1 символу
        for sym in word:
            word_with_del = word.replace(sym, '')
            wordforms.append(word_with_del)  # добавляем полученные словоформы в список
        for wordform in wordforms:
            if wordform in vocabulary2:  # если полученные словоформы есть в словаре ошибок, то добавляем их в новый список
                wordforms_keys.extend(vocabulary2[wordform])
                
        for candidate in wordforms_keys: #проходимся по потенциальным кандидатам
            if candidate not in candidates: #если потенциальный кандидат не в списке кандидатов, то
                candidates[candidate] = P(candidate) #добавляем его в словарь кандидатов
            else:
                candidates[candidate] += 1

        sorted_values = sorted(candidates, key=candidates.get)
        best_candidate.append(sorted_values[0])
        return best_candidate

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vladislavasan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladislavasan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vladislavasan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vladislavasan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
algoritm('кафка')

['кашка']

Оценим метриками качества

In [10]:

def algoritm(word):
    wordforms = []
    wordforms_keys = []
    candidates = {}
    best_candidate = []

    if word in vocabulary1:  # если слово есть в словаре норм слов, возвращаем его
        return word
    else:  # иначе – удаляем по 1 символу
        for sym in word:
            word_with_del = word.replace(sym, '')
            wordforms.append(word_with_del)  # добавляем полученные словоформы в список
        for wordform in wordforms:
            if wordform in vocabulary2:  # если полученные словоформы есть в словаре ошибок, то добавляем их в новый список
                wordforms_keys.extend(vocabulary2[wordform])

        for candidate in wordforms_keys:  # проходимся по потенциальным кандидатам
            if candidate not in candidates:  # если потенциальный кандидат не в списке кандидатов, то
                candidates[candidate] = P(candidate)  # добавляем его в словарь кандидатов
            else:
                candidates[candidate] += 1

        sorted_values = sorted(candidates, key=candidates.get)
        best_candidate.append(sorted_values)
        return best_candidate

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token) - punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token) - punct)]

    return list(zip(tokens_1, tokens_2))


vocab = set()

for sent in corpus:
    vocab.update(sent)

def predict_mistaken(word, vocab):

    if word in vocab:
        return 0
    else:
        return 1

y_true = []
y_pred = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)

        y_pred.append(predict_mistaken(pair[1], vocab))

correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], algoritm(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

    if not i % 100:
        print(i)

print(classification_report(y_true, y_pred))
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0
100
200
300
400
500
600
700
800
900
              precision    recall  f1-score   support

           0       0.98      0.86      0.92      8707
           1       0.49      0.91      0.64      1303

    accuracy                           0.86     10010
   macro avg       0.74      0.88      0.78     10010
weighted avg       0.92      0.86      0.88     10010

0.5685314685314685
0.0
0.3463879637073619


# Задание 2

## Реализация этого алгоритма с триграммной моделью

In [11]:
bad = ". ".join(open('mistakes.txt').read().splitlines()[:10])
true = ". ".join(open('correct.txt').read().splitlines()[:10])

def normalize(text):
    normalized_text = [(word.strip(punctuation)) for word \
                       in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


corpus = []
for text in open('corpus_5000.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

N = sum(WORDS.values())


def P(word, N=N):
    return WORDS[word] / N


threegramms = get_threegrams_from_text(true)

vocabulary_words_with_frequency = {}
for sent in corpus:
    for word in sent:
        if word not in vocabulary_words_with_frequency:
            vocabulary_words_with_frequency[word] = P(word)
        else:
            vocabulary_words_with_frequency[word] += 1

vocabulary_wrong_form_with_variants = {}
vocabulary_words_with_threegrams = {}
for key in vocabulary_words_with_frequency.keys():
    for sym in key:
        word_with_del = key.replace(sym, '')
        if word_with_del in vocabulary_wrong_form_with_variants:
            vocabulary_wrong_form_with_variants[word_with_del].append(key)
        else:
            vocabulary_wrong_form_with_variants[word_with_del] = [key]
    for tg in threegramms:
        try:
            if re.search(r"\b" + key + r"\b", tg) is not None:
                if key in vocabulary_words_with_threegrams:
                    vocabulary_words_with_threegrams[key].append(tg)
                else:
                    vocabulary_words_with_threegrams[key] = [tg]
        except Exception:
            # непонятно, как автоматически экранировать спецсимволы
            print('# Ключ ломает все: ' + key)
            break


def algoritm(word):
    wordforms = []
    wordforms_keys = []
    candidates = {}

    if word in vocabulary_words_with_frequency:  # если слово есть в словаре норм слов, возвращаем его
        return [word]
    else:  # иначе – удаляем по 1 символу
        for sym in word:
            word_with_del = word.replace(sym, '')
            wordforms.append(word_with_del)  # добавляем полученные словоформы в список
        for wordform in wordforms:
            if wordform in vocabulary_wrong_form_with_variants:  # если полученные словоформы есть в словаре ошибок, то добавляем их в новый список
                wordforms_keys.extend(vocabulary_wrong_form_with_variants[wordform])

        for candidate in wordforms_keys:  # проходимся по потенциальным кандидатам
            if candidate not in candidates:  # если потенциальный кандидат не в списке кандидатов, то
                candidates[candidate] = P(candidate)  # добавляем его в словарь кандидатов
            else:
                candidates[candidate] += 1

        sorted_values = sorted(candidates, key=candidates.get)
        return sorted_values


def run(text):
    correct_text = []
    words = normalize(text)
    i = 0
    len_words = len(words)
    print(words)
    while i < len_words - 2:
        local_words = [words[i], words[i + 1], words[i + 2]]
        check_if_bad_word = -1
        for index, word in enumerate(local_words):
            if word not in vocabulary_words_with_frequency:
                check_if_bad_word = index
                break
        if check_if_bad_word != -1:
            print('BAD: ' + local_words[check_if_bad_word])
            candidates = algoritm(local_words[check_if_bad_word])
            is_fixed = False
            print('Candidates: ' + str(candidates))
            for cand in candidates:
                local_words[check_if_bad_word] = cand
                tg = " ".join(local_words)
                if cand in vocabulary_words_with_threegrams and tg in vocabulary_words_with_threegrams[cand]:
                    correct_text.extend(local_words)
                    is_fixed = True
                    print("Успех! Заменили " + words[i + check_if_bad_word] + " на " + local_words[check_if_bad_word])
                    break
            if not is_fixed:
                local_words[check_if_bad_word] = words[i + check_if_bad_word]
                print('!!! Не починили ' + local_words[check_if_bad_word])
                correct_text.extend(local_words)
            i = i + 3
        else:
            correct_text.append(local_words[0])
            i = i + 1

    return " ".join(correct_text)


print(run(bad))

# Ключ ломает все: inc.(аффилированная
# Ключ ломает все: главредства(вначале
# Ключ ломает все: lofficiel),но
# Ключ ломает все: у***ном
# Ключ ломает все: u(1
# Ключ ломает все: su(3
# Ключ ломает все: su(2
# Ключ ломает все: конкордия"(верхний
# Ключ ломает все: сегодня(10.11.2018
['симпатичнейшое', 'шпионское', 'устройство', 'такой', 'себе', 'гламурный', 'фотоаппарат', 'девушки', 'бонда', 'миниатюрная', 'модель', 'камеры', 'superheadz', 'clap', 'camera', 'опофеозом', 'дня', 'для', 'меня', 'сегодня', 'стала', 'фраза', 'услышанная', 'в', 'новостях', 'пояним', 'эту', 'мысль', 'полчатся', 'вот', 'такие', 'язычки', 'в', 'массе', 'своей', 'они', 'конечно', 'все', 'оччччень', 'милые', 'нащщот', 'чавеса', 'разве', 'что', 'не', 'соглашусь', 'многие', 'сетуют', 'на', 'отсуствие', 'живого', 'взаимодействия', 'между', 'учеником', 'и', 'учителем', 'а', 'в', 'чем', 'оно', 'по', 'сути', 'основая', 'цель', 'мероприятия', 'практическая', 'отработка', 'навыков', 'по', 'оказанию', 'помощи', 'граждана

TypeError: 'NoneType' object is not subscriptable

Проверим на улучшение качества

In [12]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token) - punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token) - punct)]

    return list(zip(tokens_1, tokens_2))


vocab = set()

for sent in corpus:
    vocab.update(sent)

def predict_mistaken(word, vocab):

    if word in vocab:
        return 0
    else:
        return 1

y_true = []
y_pred = []

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if pair[0] == pair[1]:
            y_true.append(0)
        else:
            y_true.append(1)

        y_pred.append(predict_mistaken(pair[1], vocab))

correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], algoritm(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

    if not i % 100:
        print(i)

print(classification_report(y_true, y_pred))
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0
100
200
300
400
500
600
700
800
900
              precision    recall  f1-score   support

           0       0.12      0.90      0.22        77
           1       0.92      0.16      0.27       576

    accuracy                           0.25       653
   macro avg       0.52      0.53      0.25       653
weighted avg       0.83      0.25      0.27       653

0.0
0.0
1.0
